# Las Vegas Review 文本内容处理

### 该 Notebook 对应的博客[链接](http://xiehongfeng100.github.io/2018/07/31/yelper-dpps-las-vegas-data-preprocessing/)

In [1]:
import pandas as pd
import multiprocessing as mp

In [2]:
import re
import string
from nltk import word_tokenize, pos_tag
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer

In [3]:
pd.set_option('display.max_colwidth', -1)

## 1. 加载数据

In [4]:
yelp_lv_rts = pd.read_csv('../../dataset/las_vegas/review/las_vegas_review_text_with_db_id.csv')

/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
len(yelp_lv_rts)

1604247

In [6]:
yelp_lv_rts[:2]

,review_db_id,text
0,3,Pizza here made my night... Good people and great pizza. They can do anything you ask with a great attitude!
1,6,"One of my absolute favorite restaurants! I usually go to the one on Sahara but I finally moved over by UNLV and was so privileged enough to have one newly opened here! The staff are soooo nice and friendly, and the restaurant renovations are so gorgeous and such a comfortable environment! Amazing prices for amazing food!! :)"


In [7]:
yelp_lv_rts.dropna(subset=['text'], inplace=True)

## 2. 分词（Tokenize）及移除停用词（Stop Words）

In [8]:
stop_words = stopwords.words('english') + list(string.punctuation)

In [9]:
def tokenize_and_rm_stopwords(review_text):
    text_words = [
        word for word in word_tokenize(re.sub('[^a-zA-Z]', ' ', review_text).lower())
        if word not in stop_words
    ]
    return text_words

In [10]:
# 利用多进程加快计算
pool = mp.Pool(processes=12)
yelp_lv_rv_texts = pool.map(tokenize_and_rm_stopwords, yelp_lv_rts.text.values)
pool.close()
pool.join()

In [11]:
yelp_lv_rv_texts[:2]

[['pizza',
  'made',
  'night',
  'good',
  'people',
  'great',
  'pizza',
  'anything',
  'ask',
  'great',
  'attitude'],
 ['one',
  'absolute',
  'favorite',
  'restaurants',
  'usually',
  'go',
  'one',
  'sahara',
  'finally',
  'moved',
  'unlv',
  'privileged',
  'enough',
  'one',
  'newly',
  'opened',
  'staff',
  'soooo',
  'nice',
  'friendly',
  'restaurant',
  'renovations',
  'gorgeous',
  'comfortable',
  'environment',
  'amazing',
  'prices',
  'amazing',
  'food']]

## 3. 词形还原（Lemmatization）

In [12]:
lemmatizer = WordNetLemmatizer()

In [13]:
# Ref: https://stackoverflow.com/a/15590384
# Treebank tag: https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

In [14]:
def lemmatize_words(words):
    res = []
    for word, pos in pos_tag(words):
        wordnet_pos = get_wordnet_pos(pos) or wordnet.NOUN
        res.append(str(lemmatizer.lemmatize(word, pos=wordnet_pos)))
    return ' '.join(res)

In [15]:
# 利用多进程加快计算
pool = mp.Pool(processes=12)
yelp_lv_rv_lemm_texts = pool.map(lemmatize_words, yelp_lv_rv_texts)
pool.close()
pool.join()

In [16]:
yelp_lv_rv_lemm_texts[:2]

['pizza make night good people great pizza anything ask great attitude',
 'one absolute favorite restaurant usually go one sahara finally move unlv privilege enough one newly open staff soooo nice friendly restaurant renovation gorgeous comfortable environment amaze price amaze food']

## 4. 保存数据

In [17]:
yelp_lv_rts = yelp_lv_rts.assign(text_words=yelp_lv_rv_lemm_texts)

In [18]:
# 删除除 text_words 中值为 NA/NAN 的记录
yelp_lv_rts.dropna(subset=['text_words'], inplace=True)

In [19]:
# 查看 text_words 为空或非英语语种
yelp_lv_rts[yelp_lv_rts.text_words == '']

,review_db_id,text,text_words
2092,6822,蜂蜜杏仁拿铁味道很不错，不到五美金中杯。逛街累了可以来一杯^o^ 味道是有点苦，微微杏仁的苦味。还不错。服务也不错，很热情。,
7679,24818,我選擇是一個小時的全身按摩，她幫我按摩腳就佔據一大半的時間 慢慢拖延時間 按腳按的很久，終於等到要按背了 按不到五分鐘就說時間到了 下次再補時間給你 整個讓我很傻眼！！！不會再介紹朋友去了,
8925,29043,服务非常好，番茄牛腩乌冬面居然很美味，韭菜煎饺也好吃，已经出乎我的意料。红油抄手不太好吃。但是来美国十几天了，只有这有热茶。我点了白豪，添了好几次水，我们中国人还是离不开茶啊^^,
13258,43251,前回は20時ぐらいに来店したら2時間ぐらい待って席に案内されました。今回は11時ぐらいに来店したら待ちなしでした！わーいわーい\n\nオイスターは当たりはずれがあるのでハーフはオススメしません。笑っ \n味も夜の太っちょおじさんが美味しかったです。\nただ、午前中は待ち時間が少ないです。,
16874,55098,ハワイのお土産で有名なホノルルクッキー！普段はパイナップルの形ですが、ラスベガスバージョンのトランプ型もあります。味はサクサクでホワイトチョコが私のお気にいり。ラスベガスに遊びに行った時は是非！,
22304,73034,街中心から少し離れた場所にありますが、犬が一緒だったので、ここに滞在しました。部屋は清潔だしキッチンも付いていて安くて良かったです。暑かったので、プールは混んでいました。ファミリー向けの宿だと思います。,
30033,98742,韓国系の方がされている様なので完全な日本食とは違いますが、リーズナブルな値段で居酒屋や定食屋っぽいメニューも多々あり日本人が行っても満足できるのではと思います。\nストリップでは創作系の高級ジャパニーズが並ぶラスベガスで、気軽に日本食を食べるのには良い店だと思います。\n日本語を話せる店員さんはおられませんでしたが、丁寧に接客頂きました。,
35153,115474,"很乾淨的機場,等待的飛機時間長也不無聊.有很棒的吧檯餐廳還有老虎機,哈哈",
51985,171074,高いけど メチャ美味しいし 品揃えも豊富で お料理一品一品の見た目もとても綺麗です。 特に茹でたカニと ステーキ デザート達には 大大満足！,
62907,206401,日本語通じて料理も美味しかった。\n料金もリーズナブルでまた行きたいです。\nバッテラ寿しがなかったのが残念です！,


In [20]:
# 去掉 text_words 为空或非英语语种的记录
yelp_lv_rts = yelp_lv_rts[yelp_lv_rts.text_words != '']

In [21]:
yelp_lv_rts[['review_db_id', 'text_words']].to_csv('../../dataset/las_vegas/review/las_vegas_review_text_preprocessed_with_db_id.csv', index=False)